## 6_MODELING_PREDICTION
E 등급 예측 모델, S 등급 예측 모델, G 등급 예측 모델링과 결과값 예측 코드입니다. 각 분야마다 하단의 세가지 모델링을 시도하였습니다.
1) AUTOML
2) LGBM
3) RandomForest

test 결과 RandomForest의 f1-score가 모든 분야에서 가장 높게 측정되었습니다. (ESG 종합등급은 AutoML 모델의 성능이 미세하게 높습니다. 자세한 내용은 보고서를 참고부탁드립니다.) 

## 기본 설정

In [1]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

! pip install {USER_FLAG} --upgrade google-cloud-aiplatform
! pip install {USER_FLAG} --upgrade google-cloud-storage
!pip install wget

In [2]:
# 설치가 끝나면 커널을 재시작해야합니다.
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [96]:
# 세팅을 다시 로드
import os
import pandas as pd

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
import warnings
import os
import joblib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from google.cloud import aiplatform, storage

from google.cloud import storage
from google.cloud import bigquery

import pandas as pd
from sklearn import datasets

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
    
# 지역, GCS 클라이언트, BQ 클라이언트 설정
REGION = 'asia-northeast3'
gcs = storage.Client(project = PROJECT_ID)
bq = bigquery.Client(project = PROJECT_ID)

# bucket 이름 지정: "프로젝트 이름-bucket"
BUCKET_NAME=PROJECT_ID + "-bucket"
if not gcs.lookup_bucket(BUCKET_NAME):
    bucketDef = gcs.bucket(BUCKET_NAME)
    bucket = gcs.create_bucket(bucketDef, project=PROJECT_ID, location=REGION)
    print(bucket)
else:
    print(gcs.lookup_bucket(BUCKET_NAME))
    
def get_dataframe(bq_table):
    
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    project_number = shell_output[0]
    bqclient = bigquery.Client(project=project_number)
    
    table = bigquery.TableReference.from_string(
        project_number+"."+bq_table
    )
    
    rows = bqclient.list_rows(
        table
    )
    
    dataframe = rows.to_dataframe(
        create_bqstorage_client=True,
    )

    return dataframe
def to_bigquery(
    dataframe,
    table_name: str,
    bucket_name: str
):
    client = bigquery.Client()
    column_list=dataframe.columns
    query_column_list=""
    
    #blob = storage.blob.Blob.from_string(f"""gs://{bucket_name}/{table_name.split(".")[-1]}.csv""", client=storage.Client())
    dataframe.to_csv(f"""gs://{bucket_name}/{table_name.split(".")[-1]}.csv""",index=False,encoding="utf-8")
    #blob.upload_from_file(temp)
    
    for name in column_list:
        query_column_list=query_column_list+name+" STRING,"
        
    query = f"""
    LOAD DATA OVERWRITE {table_name}(
    {query_column_list}
        )
    FROM FILES (
      skip_leading_rows=1,
      allow_jagged_rows=true,
      format = 'CSV',
      encoding='utf-8',
      field_delimiter=',',
      uris = ['gs://{bucket_name}/{table_name.split(".")[-1]}.csv']);"""
    
    query_job = client.query(query)
    results = query_job.result()

Project ID:  bigfesta2022-esg-team6-proj
<Bucket: bigfesta2022-esg-team6-proj-bucket>


In [97]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [98]:
!pip install bayesian-optimization

## 0. import data

In [99]:
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score
## TRAIN 데이터 불러오기
E_train=pd.read_csv("data/pipeline3/modeling_data/E_train.csv")
S_train=pd.read_csv("data/pipeline3/modeling_data/S_train.csv")
G_train=pd.read_csv("data/pipeline3/modeling_data/G_train.csv")
ESG_train=pd.read_csv("data/pipeline3/modeling_data/ESG_train.csv")

## test 데이터 불러오기
E_test=pd.read_csv("data/pipeline3/modeling_data/E_test.csv")
S_test=pd.read_csv("data/pipeline3/modeling_data/S_test.csv")
G_test=pd.read_csv("data/pipeline3/modeling_data/G_test.csv")
ESG_test=pd.read_csv("data/pipeline3/modeling_data/ESG_test.csv")

In [100]:
def get_clf_eval(y_test, y_pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    #average를 macro로 두면 각 열에 대한 precision 값을 모두 더한 다음 열의 갯수로 나눈 것입니다.
    #즉 average를 None으로 두었을 때 구한 각 열의 Precision들을 산술 평균한 값이 macro가 됩니다.
    #average를 micro로 두면 전체 평균으로 모든 열에서 맞은 것 즉, 대각선 성분의 총 합을 총 갯수로 나눈 것입니다.
    precision = precision_score(y_test, y_pred , average = 'macro')
    recall = recall_score(y_test, y_pred, average = 'macro')
    F1 = f1_score(y_test, y_pred, average = 'macro')

    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))

## 1. E Modeling: LGBM, RandomForest, AutoML

### 1-1) E_LGBM

In [101]:
## 전체 데이터를 8 : 1 : 1 로 TRAIN, VALID, TEST로 나눠야됨
train, validate, test = np.split(E_train.sample(frac=1, random_state=48), [int(.8*len(E_train)), int(.9*len(E_train))])
x_train = train.drop(['E_GRADE'], axis = 1)
y_train = train['E_GRADE']
x_validate = validate.drop(['E_GRADE'], axis = 1)
y_validate = validate['E_GRADE']
x_test = test.drop(['E_GRADE'], axis = 1)
y_test = test['E_GRADE']

In [102]:
# LGBM의 하이퍼 파라미터의 범위를 dictionary 형태로 지정
lgbm_parameter_bounds = {'n_estimators' : (100,400), 
                         'num_leaves' : (16,40), 
                         'learning_rate' : (0.05 , 0.3), 
                         'early_stopping_rounds' : (100 , 100),
                        }

def lgbm_bo(n_estimators, num_leaves, learning_rate, early_stopping_rounds):
    lgbm_params = {'n_estimators' : int(round(n_estimators)),'num_leaves' : int(round(num_leaves)), 'learning_rate' : float(learning_rate), 'early_stopping_rounds' : int(round(early_stopping_rounds))}
    lgbm = LGBMClassifier(**lgbm_params, application = 'multiclass' , random_state = 48)
    evals = [(x_validate, y_validate)]
    lgbm.fit(x_train,y_train, eval_metric='multi_logloss', eval_set=evals)
    F1_score = f1_score(y_test, lgbm.predict(x_test), average = 'macro')

    return F1_score



# 이제 Bayesian Optimization을 사용할 준비가 끝났습니다.

# "BO_lgbm"라는 변수에 Bayesian Optmization을 저장해보세요

BO_lgbm = BayesianOptimization(f = lgbm_bo, pbounds = lgbm_parameter_bounds, random_state = 48)



# Bayesian Optimization을 실행해보세요

BO_lgbm.maximize(init_points = 5, n_iter = 30)

|   iter    |  target   | early_... | learni... | n_esti... | num_le... |
-------------------------------------------------------------------------
[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
[1]	valid_0's multi_logloss: 1.64458
[2]	valid_0's multi_logloss: 1.42358
[3]	valid_0's multi_logloss: 1.28719
[4]	valid_0's multi_logloss: 1.21336
[5]	valid_0's multi_logloss: 1.16118
[6]	valid_0's multi_logloss: 1.10685
[7]	valid_0's multi_logloss: 1.08358
[8]	valid_0's multi_logloss: 1.05464
[9]	valid_0's multi_logloss: 1.03004
[10]	valid_0's multi_logloss: 0.995052
[11]	valid_0's multi_logloss: 0.975891
[12]	valid_0's multi_logloss: 0.954383
[13]	valid_0's multi_logloss: 0.930304
[14]	valid_0's multi_logloss: 0.922095
[15]	valid_0's multi_logloss: 0.913748
[16]	valid_0's multi_logloss: 0.902678
[17]	valid_0's multi_logloss: 0.906946
[18]	valid_0's multi_logloss: 0.898367
[19]	valid_0's multi_logloss: 0

/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.p

[17]	valid_0's multi_logloss: 1.10318
[18]	valid_0's multi_logloss: 1.08985
[19]	valid_0's multi_logloss: 1.0788
[20]	valid_0's multi_logloss: 1.06341
[21]	valid_0's multi_logloss: 1.05056
[22]	valid_0's multi_logloss: 1.03645
[23]	valid_0's multi_logloss: 1.03182
[24]	valid_0's multi_logloss: 1.01922
[25]	valid_0's multi_logloss: 1.00671
[26]	valid_0's multi_logloss: 0.998815
[27]	valid_0's multi_logloss: 0.993386
[28]	valid_0's multi_logloss: 0.989073
[29]	valid_0's multi_logloss: 0.985191
[30]	valid_0's multi_logloss: 0.980048
[31]	valid_0's multi_logloss: 0.971982
[32]	valid_0's multi_logloss: 0.966268
[33]	valid_0's multi_logloss: 0.964118
[34]	valid_0's multi_logloss: 0.961183
[35]	valid_0's multi_logloss: 0.954826
[36]	valid_0's multi_logloss: 0.950459
[37]	valid_0's multi_logloss: 0.945812
[38]	valid_0's multi_logloss: 0.942169
[39]	valid_0's multi_logloss: 0.935782
[40]	valid_0's multi_logloss: 0.933833
[41]	valid_0's multi_logloss: 0.930873
[42]	valid_0's multi_logloss: 0.926

/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.p

[1]	valid_0's multi_logloss: 1.68657
[2]	valid_0's multi_logloss: 1.48796
[3]	valid_0's multi_logloss: 1.36527
[4]	valid_0's multi_logloss: 1.28005
[5]	valid_0's multi_logloss: 1.22123
[6]	valid_0's multi_logloss: 1.16681
[7]	valid_0's multi_logloss: 1.12262
[8]	valid_0's multi_logloss: 1.08392
[9]	valid_0's multi_logloss: 1.04651
[10]	valid_0's multi_logloss: 1.0113
[11]	valid_0's multi_logloss: 0.986575
[12]	valid_0's multi_logloss: 0.960361
[13]	valid_0's multi_logloss: 0.943745
[14]	valid_0's multi_logloss: 0.930093
[15]	valid_0's multi_logloss: 0.920159
[16]	valid_0's multi_logloss: 0.916459
[17]	valid_0's multi_logloss: 0.91079
[18]	valid_0's multi_logloss: 0.900347
[19]	valid_0's multi_logloss: 0.894184
[20]	valid_0's multi_logloss: 0.892835
[21]	valid_0's multi_logloss: 0.895117
[22]	valid_0's multi_logloss: 0.898269
[23]	valid_0's multi_logloss: 0.905034
[24]	valid_0's multi_logloss: 0.903081
[25]	valid_0's multi_logloss: 0.906588
[26]	valid_0's multi_logloss: 0.903531
[27]	va

...

[11]	valid_0's multi_logloss: 1.02272
[12]	valid_0's multi_logloss: 1.00917
[13]	valid_0's multi_logloss: 0.996139
[14]	valid_0's multi_logloss: 0.987111
[15]	valid_0's multi_logloss: 0.978366
[16]	valid_0's multi_logloss: 0.968277
[17]	valid_0's multi_logloss: 0.961666
[18]	valid_0's multi_logloss: 0.959451
[19]	valid_0's multi_logloss: 0.957351
[20]	valid_0's multi_logloss: 0.957463
[21]	valid_0's multi_logloss: 0.955468
[22]	valid_0's multi_logloss: 0.959278
[23]	valid_0's multi_logloss: 0.958557
[24]	valid_0's multi_logloss: 0.955675
[25]	valid_0's multi_logloss: 0.96067
[26]	valid_0's multi_logloss: 0.951095
[27]	valid_0's multi_logloss: 0.953715
[28]	valid_0's multi_logloss: 0.958839
[29]	valid_0's multi_logloss: 0.96193
[30]	valid_0's multi_logloss: 0.958293
[31]	valid_0's multi_logloss: 0.95937
[32]	valid_0's multi_logloss: 0.9607
[33]	valid_0's multi_logloss: 0.960233
[34]	valid_0's multi_logloss: 0.966291
[35]	valid_0's multi_logloss: 0.962772
[36]	valid_0's multi_logloss: 0.

/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.p

[6]	valid_0's multi_logloss: 1.27119
[7]	valid_0's multi_logloss: 1.24088
[8]	valid_0's multi_logloss: 1.20082
[9]	valid_0's multi_logloss: 1.17406
[10]	valid_0's multi_logloss: 1.13415
[11]	valid_0's multi_logloss: 1.10552
[12]	valid_0's multi_logloss: 1.07645
[13]	valid_0's multi_logloss: 1.05266
[14]	valid_0's multi_logloss: 1.03002
[15]	valid_0's multi_logloss: 1.01039
[16]	valid_0's multi_logloss: 0.991677
[17]	valid_0's multi_logloss: 0.978895
[18]	valid_0's multi_logloss: 0.974186
[19]	valid_0's multi_logloss: 0.972305
[20]	valid_0's multi_logloss: 0.96173
[21]	valid_0's multi_logloss: 0.955641
[22]	valid_0's multi_logloss: 0.952506
[23]	valid_0's multi_logloss: 0.949841
[24]	valid_0's multi_logloss: 0.9477
[25]	valid_0's multi_logloss: 0.946065
[26]	valid_0's multi_logloss: 0.945091
[27]	valid_0's multi_logloss: 0.944716
[28]	valid_0's multi_logloss: 0.940075
[29]	valid_0's multi_logloss: 0.943482
[30]	valid_0's multi_logloss: 0.946381
[31]	valid_0's multi_logloss: 0.945394
[32

/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.p

[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
[1]	valid_0's multi_logloss: 1.67025
[2]	valid_0's multi_logloss: 1.4689
[3]	valid_0's multi_logloss: 1.33408
[4]	valid_0's multi_logloss: 1.24203
[5]	valid_0's multi_logloss: 1.18356
[6]	valid_0's multi_logloss: 1.11928
[7]	valid_0's multi_logloss: 1.08594
[8]	valid_0's multi_logloss: 1.04711
[9]	valid_0's multi_logloss: 1.02317
[10]	valid_0's multi_logloss: 0.996303
[11]	valid_0's multi_logloss: 0.976507
[12]	valid_0's multi_logloss: 0.971904
[13]	valid_0's multi_logloss: 0.951399
[14]	valid_0's multi_logloss: 0.945233
[15]	valid_0's multi_logloss: 0.940635
[16]	valid_0's multi_logloss: 0.944055
[17]	valid_0's multi_logloss: 0.935868
[18]	valid_0's multi_logloss: 0.933265
[19]	valid_0's multi_logloss: 0.939258
[20]	valid_0's multi_logloss: 0.944035
[21]	valid_0's multi_logloss: 0.94211
[22]	valid_0's multi_logloss: 0.938073
[23]	valid_0's multi_logl

In [103]:
max_params = BO_lgbm.max['params']
max_params['learning_rate'] = float(max_params['learning_rate'])
max_params['n_estimators'] = int(max_params['n_estimators'])
max_params['num_leaves'] = int(max_params['num_leaves'])
print(max_params)

{'early_stopping_rounds': 100.0, 'learning_rate': 0.14267473281927506, 'n_estimators': 398, 'num_leaves': 17}


In [104]:
E_lgbm_tune =LGBMClassifier(n_estimators = 398 , learning_rate = 0.143, early_stopping_rounds = 100, num_leaves = 17)
evals = [(x_validate, y_validate)]
E_lgbm_tune.fit(x_train,y_train, eval_metric='multi_logloss', eval_set=evals)

[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
[1]	valid_0's multi_logloss: 1.85754
[2]	valid_0's multi_logloss: 1.68181
[3]	valid_0's multi_logloss: 1.55872
[4]	valid_0's multi_logloss: 1.47338
[5]	valid_0's multi_logloss: 1.40674
[6]	valid_0's multi_logloss: 1.3455
[7]	valid_0's multi_logloss: 1.29492
[8]	valid_0's multi_logloss: 1.2572
[9]	valid_0's multi_logloss: 1.21995
[10]	valid_0's multi_logloss: 1.19274
[11]	valid_0's multi_logloss: 1.16792
[12]	valid_0's multi_logloss: 1.1386
[13]	valid_0's multi_logloss: 1.10973
[14]	valid_0's multi_logloss: 1.08864
[15]	valid_0's multi_logloss: 1.07849
[16]	valid_0's multi_logloss: 1.06912
[17]	valid_0's multi_logloss: 1.05512
[18]	valid_0's multi_logloss: 1.04196
[19]	valid_0's multi_logloss: 1.02804
[20]	valid_0's multi_logloss: 1.02054
[21]	valid_0's multi_logloss: 1.00766
[22]	valid_0's multi_logloss: 0.9961
[23]	valid_0's multi_logloss: 0.993376
[

LGBMClassifier(early_stopping_rounds=100, learning_rate=0.143, n_estimators=398,
               num_leaves=17)

In [105]:
x_test_new = pd.concat([x_validate, x_test])
y_test_new = pd.concat([y_validate, y_test])

In [106]:
pred = E_lgbm_tune.predict(x_test_new)

In [107]:
get_clf_eval(y_test_new, pred)

오차행렬:
 [[ 5  0  0  0  0  0  0  0  0]
 [ 4 10  0  0  0  0  2  0  0]
 [ 0  0  6  0  0  1  1  1  0]
 [ 2  0  3 19  0  2  0  0  1]
 [ 0  0  0  0  8  9  1  1  4]
 [ 0  0  2  0  0 23  7  4  3]
 [ 0  0  0  0  5  3 13  5  3]
 [ 0  0  0  1  2  2  3 13  0]
 [ 1  0  0  0  1  2  8  6 35]]

정확도: 0.5946
정밀도: 0.6139
재현율: 0.6290
F1: 0.5964


In [108]:
confusion = confusion_matrix(y_test_new, pred)

val = 0
for i in range(0,9):
    for j in range(0,9):
        val = val + confusion[i][j] * (((j+1) - (i+1))^2) 
print(val / sum(sum(confusion)))

1.1486486486486487


### 1-2) RandomForest

In [109]:
train, test = np.split(E_train.sample(frac=1, random_state=48), [int(.8*len(E_train))])

In [110]:
x_train = train.drop(['E_GRADE'], axis = 1)
y_train = train['E_GRADE']
x_test = test.drop(['E_GRADE'], axis = 1)
y_test = test['E_GRADE']

In [111]:
# 랜덤 포레스트의 하이퍼 파라미터의 범위를 dictionary 형태로 지정

RF_parameter_bounds = {'n_estimators' : (5,15), 'max_depth' : (1,50), 'min_samples_leaf' : (1, 10), 'min_samples_split' : (2,10)}

def RF_bo(n_estimators, max_depth, min_samples_leaf, min_samples_split):
    RF_params = {'n_estimators' : int(round(n_estimators)),
                 'max_depth' : int(round(max_depth)), 
                 'min_samples_leaf' : int(round(min_samples_leaf)),
                 'min_samples_split' : int(round(min_samples_split))}
    RF =  RandomForestClassifier(**RF_params, random_state = 48)
    RF.fit(x_train,y_train)
    F1_score = f1_score(y_test, RF.predict(x_test), average = 'macro')

    return F1_score



# 이제 Bayesian Optimization을 사용할 준비가 끝났습니다.

# "BO_lgbm"라는 변수에 Bayesian Optmization을 저장해보세요

BO_RF = BayesianOptimization(f = RF_bo, pbounds = RF_parameter_bounds, random_state = 48)



# Bayesian Optimization을 실행해보세요

BO_RF.maximize(init_points = 5, n_iter = 50)

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        |  0.1612   |  1.857    |  9.024    |  4.279    |  7.99     |
|  2        |  0.4956   |  39.81    |  3.92     |  8.918    |  9.475    |
|  3        |  0.4532   |  27.86    |  4.215    |  2.898    |  6.419    |
|  4        |  0.4017   |  22.8     |  7.588    |  5.681    |  10.93    |
|  5        |  0.4714   |  17.5     |  5.09     |  3.497    |  9.088    |
|  6        |  0.4901   |  40.64    |  3.829    |  9.171    |  8.315    |
|  7        |  0.6192   |  49.07    |  1.0      |  2.0      |  15.0     |
|  8        |  0.4174   |  49.91    |  8.835    |  2.523    |  14.57    |
|  9        |  0.6192   |  42.05    |  1.0      |  2.0      |  15.0     |
|  10       |  0.6003   |  50.0     |  1.0      |  2.0      |  5.0      |
|  11       |  0.5715   |  50.0     |  1.0      |  10.0     |  15.0     |
|  12       |  0.6003   |  41.26    | 

In [112]:
max_params = BO_RF.max['params']
max_params['n_estimators'] = int(max_params['n_estimators'])
max_params['max_depth'] = int(max_params['max_depth'])
max_params['min_samples_leaf'] = int(max_params['min_samples_leaf'])
max_params['min_samples_split'] = int(max_params['min_samples_split'])
print(max_params)

{'max_depth': 17, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 12}


In [113]:
E_RF_tune =  RandomForestClassifier(max_depth = 17, min_samples_leaf = 1, n_estimators = 12, min_samples_split = 2, random_state = 48)
E_RF_tune.fit(x_train,y_train)

pred = E_RF_tune.predict(x_test)

In [114]:
get_clf_eval(y_test, pred)

오차행렬:
 [[ 5  0  0  0  0  0  0  0  0]
 [ 4 10  0  0  0  0  2  0  0]
 [ 0  0  6  1  0  0  1  1  0]
 [ 2  0  1 20  3  1  0  0  0]
 [ 0  0  0  0 15  4  0  1  3]
 [ 0  0  2  1  0 28  2  4  2]
 [ 0  0  0  2  3  6 12  4  2]
 [ 0  0  0  3  2  2  2 12  0]
 [ 1  0  0  0  2  6  9  4 31]]

정확도: 0.6261
정밀도: 0.6362
재현율: 0.6636
F1: 0.6282


In [115]:
confusion = confusion_matrix(y_test, pred)

val = 0
for i in range(0,9):
    for j in range(0,9):
        val = val + confusion[i][j] * (((j+1) - (i+1))^2) 
print(val / sum(sum(confusion)))

1.0225225225225225


### 1-3) AutoML

In [116]:
from google.cloud import aiplatform
import json
E_endpoint = aiplatform.Endpoint(
    endpoint_name=f"projects/{PROJECT_ID}/locations/asia-northeast3/endpoints/1067036452338008064"
)
x_test_automl=x_test.astype(str)
response = E_endpoint.predict(json.loads(x_test_automl.to_json(orient='records')))
pred_ml=[]
for row in response[0]:
    scores=row["scores"]
    for i,value in enumerate(scores):
        if value==max(row["scores"]):
            #print(row.index[i])
            pred_ml.append(int(row["classes"][i].replace("'","")))
get_clf_eval(y_test, pred_ml)

오차행렬:
 [[ 5  0  0  0  0  0  0  0  0]
 [ 4  9  1  1  0  1  0  0  0]
 [ 0  2  6  0  0  0  0  1  0]
 [ 3  2  2 15  0  0  0  0  5]
 [ 0  1  1  0  8  1  0  0 12]
 [ 0  1  4  1  1 15  1  1 15]
 [ 0  0  0  0  2  8  6  3 10]
 [ 0  0  0  1  2  2  2  5  9]
 [ 2  2  1  0  0  1  0  0 47]]

정확도: 0.5225
정밀도: 0.5464
재현율: 0.5388
F1: 0.4879


In [117]:
confusion = confusion_matrix(y_test, pred_ml)

val = 0
for i in range(0,9):
    for j in range(0,9):
        val = val + confusion[i][j] * (((j+1) - (i+1))^2) 
print(val / sum(sum(confusion)))

1.2522522522522523


## 2. S Modeling: LGBM, RandomForest, AutoML

### 2-1) LGBM

In [118]:
## 전체 데이터를 8 : 1 : 1 로 TRAIN, VALID, TEST로 나눠야됨

train, validate, test = np.split(S_train.sample(frac=1, random_state=48), [int(.8*len(S_train)), int(.9*len(S_train))])

In [119]:
x_train = train.drop(['S_GRADE'], axis = 1)
y_train = train['S_GRADE']
x_validate = validate.drop(['S_GRADE'], axis = 1)
y_validate = validate['S_GRADE']
x_test = test.drop(['S_GRADE'], axis = 1)
y_test = test['S_GRADE']

In [120]:
# LGBM의 하이퍼 파라미터의 범위를 dictionary 형태로 지정

lgbm_parameter_bounds = {'n_estimators' : (100,400), 
                         'num_leaves' : (16,40), 
                         'learning_rate' : (0.05 , 0.3), 
                         'early_stopping_rounds' : (100 , 100),
                        }

def lgbm_bo(n_estimators, num_leaves, learning_rate, early_stopping_rounds):
    lgbm_params = {'n_estimators' : int(round(n_estimators)),'num_leaves' : int(round(num_leaves)), 'learning_rate' : float(learning_rate), 'early_stopping_rounds' : int(round(early_stopping_rounds))}
    lgbm = LGBMClassifier(**lgbm_params, application = 'multiclass' , random_state = 48)
    evals = [(x_validate, y_validate)]
    lgbm.fit(x_train,y_train, eval_metric='multi_logloss', eval_set=evals)
    F1_score = f1_score(y_test, lgbm.predict(x_test), average = 'macro')

    return F1_score



# 이제 Bayesian Optimization을 사용할 준비가 끝났습니다.

# "BO_lgbm"라는 변수에 Bayesian Optmization을 저장해보세요

BO_lgbm = BayesianOptimization(f = lgbm_bo, pbounds = lgbm_parameter_bounds, random_state = 48)



# Bayesian Optimization을 실행해보세요

BO_lgbm.maximize(init_points = 5, n_iter = 30)

|   iter    |  target   | early_... | learni... | n_esti... | num_le... |
-------------------------------------------------------------------------
[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
[1]	valid_0's multi_logloss: 1.80878
[2]	valid_0's multi_logloss: 1.62438
[3]	valid_0's multi_logloss: 1.51892
[4]	valid_0's multi_logloss: 1.44048
[5]	valid_0's multi_logloss: 1.3832
[6]	valid_0's multi_logloss: 1.32169
[7]	valid_0's multi_logloss: 1.28071
[8]	valid_0's multi_logloss: 1.23205
[9]	valid_0's multi_logloss: 1.18751
[10]	valid_0's multi_logloss: 1.13029
[11]	valid_0's multi_logloss: 1.10491
[12]	valid_0's multi_logloss: 1.07537
[13]	valid_0's multi_logloss: 1.04362
[14]	valid_0's multi_logloss: 1.01657
[15]	valid_0's multi_logloss: 0.992102
[16]	valid_0's multi_logloss: 0.989743
[17]	valid_0's multi_logloss: 0.982168
[18]	valid_0's multi_logloss: 0.974857
[19]	valid_0's multi_logloss: 0.96398

/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.p

[3]	valid_0's multi_logloss: 1.56431
[4]	valid_0's multi_logloss: 1.48166
[5]	valid_0's multi_logloss: 1.42402
[6]	valid_0's multi_logloss: 1.36753
[7]	valid_0's multi_logloss: 1.32516
[8]	valid_0's multi_logloss: 1.27843
[9]	valid_0's multi_logloss: 1.23041
[10]	valid_0's multi_logloss: 1.184
[11]	valid_0's multi_logloss: 1.14125
[12]	valid_0's multi_logloss: 1.11007
[13]	valid_0's multi_logloss: 1.08517
[14]	valid_0's multi_logloss: 1.06998
[15]	valid_0's multi_logloss: 1.05347
[16]	valid_0's multi_logloss: 1.03201
[17]	valid_0's multi_logloss: 1.01927
[18]	valid_0's multi_logloss: 1.00006
[19]	valid_0's multi_logloss: 0.984039
[20]	valid_0's multi_logloss: 0.966262
[21]	valid_0's multi_logloss: 0.962293
[22]	valid_0's multi_logloss: 0.951329
[23]	valid_0's multi_logloss: 0.942438
[24]	valid_0's multi_logloss: 0.93962
[25]	valid_0's multi_logloss: 0.928182
[26]	valid_0's multi_logloss: 0.928477
[27]	valid_0's multi_logloss: 0.928672
[28]	valid_0's multi_logloss: 0.941204
[29]	valid_0

/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.p

[5]	valid_0's multi_logloss: 1.42815
[6]	valid_0's multi_logloss: 1.3617
[7]	valid_0's multi_logloss: 1.33902
[8]	valid_0's multi_logloss: 1.29155
[9]	valid_0's multi_logloss: 1.24815
[10]	valid_0's multi_logloss: 1.19496
[11]	valid_0's multi_logloss: 1.16073
[12]	valid_0's multi_logloss: 1.12445
[13]	valid_0's multi_logloss: 1.10978
[14]	valid_0's multi_logloss: 1.09156
[15]	valid_0's multi_logloss: 1.07737
[16]	valid_0's multi_logloss: 1.07977
[17]	valid_0's multi_logloss: 1.06215
[18]	valid_0's multi_logloss: 1.04265
[19]	valid_0's multi_logloss: 1.03406
[20]	valid_0's multi_logloss: 1.02153
[21]	valid_0's multi_logloss: 1.01087
[22]	valid_0's multi_logloss: 1.00711
[23]	valid_0's multi_logloss: 0.999787
[24]	valid_0's multi_logloss: 0.995156
[25]	valid_0's multi_logloss: 0.993918
[26]	valid_0's multi_logloss: 0.994922
[27]	valid_0's multi_logloss: 0.999459
[28]	valid_0's multi_logloss: 0.994427
[29]	valid_0's multi_logloss: 1.00482
[30]	valid_0's multi_logloss: 1.00772
[31]	valid_0

...

[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
[1]	valid_0's multi_logloss: 1.85754
[2]	valid_0's multi_logloss: 1.69598
[3]	valid_0's multi_logloss: 1.60113
[4]	valid_0's multi_logloss: 1.5263
[5]	valid_0's multi_logloss: 1.46622
[6]	valid_0's multi_logloss: 1.40521
[7]	valid_0's multi_logloss: 1.3508
[8]	valid_0's multi_logloss: 1.30066
[9]	valid_0's multi_logloss: 1.24798
[10]	valid_0's multi_logloss: 1.19625
[11]	valid_0's multi_logloss: 1.16095
[12]	valid_0's multi_logloss: 1.13199
[13]	valid_0's multi_logloss: 1.10793
[14]	valid_0's multi_logloss: 1.09095
[15]	valid_0's multi_logloss: 1.07139
[16]	valid_0's multi_logloss: 1.0527
[17]	valid_0's multi_logloss: 1.03572
[18]	valid_0's multi_logloss: 1.01646
[19]	valid_0's multi_logloss: 1.00544
[20]	valid_0's multi_logloss: 0.996967
[21]	valid_0's multi_logloss: 0.986541
[22]	valid_0's multi_logloss: 0.976812
[23]	valid_0's multi_logloss: 0.9683

/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.p

[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
[1]	valid_0's multi_logloss: 1.92742
[2]	valid_0's multi_logloss: 1.76761
[3]	valid_0's multi_logloss: 1.66903
[4]	valid_0's multi_logloss: 1.61075
[5]	valid_0's multi_logloss: 1.55363
[6]	valid_0's multi_logloss: 1.50171
[7]	valid_0's multi_logloss: 1.45028
[8]	valid_0's multi_logloss: 1.417
[9]	valid_0's multi_logloss: 1.37291
[10]	valid_0's multi_logloss: 1.33086
[11]	valid_0's multi_logloss: 1.29706
[12]	valid_0's multi_logloss: 1.25856
[13]	valid_0's multi_logloss: 1.22321
[14]	valid_0's multi_logloss: 1.191
[15]	valid_0's multi_logloss: 1.16189
[16]	valid_0's multi_logloss: 1.14058
[17]	valid_0's multi_logloss: 1.10909
[18]	valid_0's multi_logloss: 1.08319
[19]	valid_0's multi_logloss: 1.06795
[20]	valid_0's multi_logloss: 1.04895
[21]	valid_0's multi_logloss: 1.0345
[22]	valid_0's multi_logloss: 1.02121
[23]	valid_0's multi_logloss: 0.998266
[2

/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.p

[3]	valid_0's multi_logloss: 1.53921
[4]	valid_0's multi_logloss: 1.45908
[5]	valid_0's multi_logloss: 1.39433
[6]	valid_0's multi_logloss: 1.3299
[7]	valid_0's multi_logloss: 1.2754
[8]	valid_0's multi_logloss: 1.23034
[9]	valid_0's multi_logloss: 1.18333
[10]	valid_0's multi_logloss: 1.14465
[11]	valid_0's multi_logloss: 1.1114
[12]	valid_0's multi_logloss: 1.07298
[13]	valid_0's multi_logloss: 1.04601
[14]	valid_0's multi_logloss: 1.02211
[15]	valid_0's multi_logloss: 0.996164
[16]	valid_0's multi_logloss: 0.980219
[17]	valid_0's multi_logloss: 0.971684
[18]	valid_0's multi_logloss: 0.971756
[19]	valid_0's multi_logloss: 0.949362
[20]	valid_0's multi_logloss: 0.940429
[21]	valid_0's multi_logloss: 0.92776
[22]	valid_0's multi_logloss: 0.922997
[23]	valid_0's multi_logloss: 0.919532
[24]	valid_0's multi_logloss: 0.909718
[25]	valid_0's multi_logloss: 0.924164
[26]	valid_0's multi_logloss: 0.935372
[27]	valid_0's multi_logloss: 0.932754
[28]	valid_0's multi_logloss: 0.927067
[29]	vali

In [121]:
max_params = BO_lgbm.max['params']
max_params['learning_rate'] = float(max_params['learning_rate'])
max_params['n_estimators'] = int(max_params['n_estimators'])
max_params['num_leaves'] = int(max_params['num_leaves'])
print(max_params)

{'early_stopping_rounds': 100.0, 'learning_rate': 0.08351276961770764, 'n_estimators': 314, 'num_leaves': 36}


In [122]:
S_lgbm_tune =LGBMClassifier(n_estimators = 314 , learning_rate = 0.08, early_stopping_rounds = 100, num_leaves = 36)
evals = [(x_validate, y_validate)]
S_lgbm_tune.fit(x_train,y_train, eval_metric='multi_logloss', eval_set=evals)

[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
[1]	valid_0's multi_logloss: 2.05381
[2]	valid_0's multi_logloss: 1.94837
[3]	valid_0's multi_logloss: 1.86932
[4]	valid_0's multi_logloss: 1.8034
[5]	valid_0's multi_logloss: 1.74745
[6]	valid_0's multi_logloss: 1.70137
[7]	valid_0's multi_logloss: 1.66261
[8]	valid_0's multi_logloss: 1.62488
[9]	valid_0's multi_logloss: 1.59545
[10]	valid_0's multi_logloss: 1.56474
[11]	valid_0's multi_logloss: 1.52757
[12]	valid_0's multi_logloss: 1.5012
[13]	valid_0's multi_logloss: 1.47572
[14]	valid_0's multi_logloss: 1.44969
[15]	valid_0's multi_logloss: 1.42235
[16]	valid_0's multi_logloss: 1.40234
[17]	valid_0's multi_logloss: 1.38426
[18]	valid_0's multi_logloss: 1.36821
[19]	valid_0's multi_logloss: 1.34307
[20]	valid_0's multi_logloss: 1.32007
[21]	valid_0's multi_logloss: 1.29579
[22]	valid_0's multi_logloss: 1.27554
[23]	valid_0's multi_logloss: 1.2635
[

LGBMClassifier(early_stopping_rounds=100, learning_rate=0.08, n_estimators=314,
               num_leaves=36)

In [123]:
x_test_new = pd.concat([x_validate , x_test])
y_test_new = pd.concat([y_validate , y_test])

In [124]:
pred = S_lgbm_tune.predict(x_test_new)

In [125]:
get_clf_eval(y_test_new, pred)

오차행렬:
 [[ 5  2  0  0  0  0  0  0  0]
 [ 0 21  0  0  0  0  0  0  0]
 [ 0  0 24  1  0  0  3  0  0]
 [ 0  0  2 11  2  0  0  0  0]
 [ 0  0  0  0 21  2  0  2  1]
 [ 0  0  0  0  1 14  2  1  1]
 [ 0  1  1  1  2  0 26  2  2]
 [ 0  0  0  3  0  8  5 14  9]
 [ 0  1  0  0  2  2  2  9 16]]

정확도: 0.6847
정밀도: 0.7156
재현율: 0.7168
F1: 0.7093


In [126]:
confusion = confusion_matrix(y_test_new, pred)

val = 0
for i in range(0,9):
    for j in range(0,9):
        val = val + confusion[i][j] * (((j+1) - (i+1))^2) 
print(val / sum(sum(confusion)))

1.1936936936936937


### 2-2) RandomForest

In [127]:
## 전체 데이터를 8 : 2 로 TRAIN, TEST로 나눠야됨

train, test = np.split(S_train.sample(frac=1, random_state=48), [int(.8*len(S_train))])

In [128]:
x_train = train.drop(['S_GRADE'], axis = 1)
y_train = train['S_GRADE']
x_test = test.drop(['S_GRADE'], axis = 1)
y_test = test['S_GRADE']

In [129]:
# 랜덤 포레스트의 하이퍼 파라미터의 범위를 dictionary 형태로 지정

RF_parameter_bounds = {'n_estimators' : (5,15), 'max_depth' : (1,50), 'min_samples_leaf' : (1, 10), 'min_samples_split' : (2,10)}

def RF_bo(n_estimators, max_depth, min_samples_leaf, min_samples_split):
    RF_params = {'n_estimators' : int(round(n_estimators)),
                 'max_depth' : int(round(max_depth)), 
                 'min_samples_leaf' : int(round(min_samples_leaf)),
                 'min_samples_split' : int(round(min_samples_split))}
    RF =  RandomForestClassifier(**RF_params, random_state = 48)
    RF.fit(x_train,y_train)
    F1_score = f1_score(y_test, RF.predict(x_test), average = 'macro')

    return F1_score



# 이제 Bayesian Optimization을 사용할 준비가 끝났습니다.

# "BO_lgbm"라는 변수에 Bayesian Optmization을 저장해보세요

BO_RF = BayesianOptimization(f = RF_bo, pbounds = RF_parameter_bounds, random_state = 48)



# Bayesian Optimization을 실행해보세요

BO_RF.maximize(init_points = 5, n_iter = 50)

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        |  0.1365   |  1.857    |  9.024    |  4.279    |  7.99     |
|  2        |  0.5049   |  39.81    |  3.92     |  8.918    |  9.475    |
|  3        |  0.4393   |  27.86    |  4.215    |  2.898    |  6.419    |
|  4        |  0.3613   |  22.8     |  7.588    |  5.681    |  10.93    |
|  5        |  0.4405   |  17.5     |  5.09     |  3.497    |  9.088    |
|  6        |  0.5361   |  49.94    |  3.114    |  5.928    |  9.33     |
|  7        |  0.3391   |  45.25    |  10.0     |  2.0      |  5.0      |
|  8        |  0.3288   |  5.224    |  7.28     |  9.798    |  7.824    |
|  9        |  0.5638   |  45.98    |  1.0      |  9.343    |  11.34    |
|  10       |  0.6967   |  44.87    |  1.0      |  2.292    |  15.0     |
|  11       |  0.6967   |  37.86    |  1.0      |  2.0      |  15.0     |
|  12       |  0.4687   |  41.15    | 

In [130]:
max_params = BO_RF.max['params']
max_params['n_estimators'] = int(max_params['n_estimators'])
max_params['max_depth'] = int(max_params['max_depth'])
max_params['min_samples_leaf'] = int(max_params['min_samples_leaf'])
max_params['min_samples_split'] = int(max_params['min_samples_split'])
print(max_params)

{'max_depth': 38, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 9}


In [131]:
S_RF_tune =  RandomForestClassifier(max_depth = 38, min_samples_leaf = 1, n_estimators = 9, min_samples_split = 4, random_state = 48)
S_RF_tune.fit(x_train,y_train)

pred = S_RF_tune.predict(x_test)

In [132]:
get_clf_eval(y_test, pred)

오차행렬:
 [[ 5  2  0  0  0  0  0  0  0]
 [ 0 21  0  0  0  0  0  0  0]
 [ 0  0 23  1  0  0  2  0  2]
 [ 0  2  2 10  1  0  0  0  0]
 [ 0  0  0  0 21  2  0  2  1]
 [ 0  0  0  0  3 16  0  0  0]
 [ 0  0  3  1  1  0 25  3  2]
 [ 0  1  0  0  0  1  9 15 13]
 [ 0  0  1  0  0  2  2 11 16]]

정확도: 0.6847
정밀도: 0.7351
재현율: 0.7168
F1: 0.7201


In [133]:
confusion = confusion_matrix(y_test, pred)

val = 0
for i in range(0,9):
    for j in range(0,9):
        val = val + confusion[i][j] * (((j+1) - (i+1))^2) 
print(val / sum(sum(confusion)))

1.2342342342342343


### 2-3) AutoML

In [134]:
from google.cloud import aiplatform
import json
S_endpoint = aiplatform.Endpoint(
    endpoint_name=f"projects/{PROJECT_ID}/locations/asia-northeast3/endpoints/6991521762143895552"
)
x_test_automl=x_test.astype(str)
response = S_endpoint.predict(json.loads(x_test_automl.to_json(orient='records')))
pred_ml=[]
for row in response[0]:
    scores=row["scores"]
    for i,value in enumerate(scores):
        if value==max(row["scores"]):
            #print(row.index[i])
            pred_ml.append(int(row["classes"][i].replace("'","")))
get_clf_eval(y_test, pred_ml)

오차행렬:
 [[ 4  2  0  0  0  0  1  0  0]
 [ 1 15  4  0  0  0  0  1  0]
 [ 0  5 18  0  2  0  1  2  0]
 [ 0  8  1  1  3  0  1  0  1]
 [ 0  1  2  0  3  0  8  6  6]
 [ 1  1  1  0  1  4  4  3  4]
 [ 0  1  2  0  2  0 15 11  4]
 [ 0  0  2  0  0  1  3 19 14]
 [ 0  0  0  0  0  0  3  9 20]]

정확도: 0.4459
정밀도: 0.5546
재현율: 0.4291
F1: 0.4167


In [135]:
confusion = confusion_matrix(y_test, pred_ml)

val = 0
for i in range(0,9):
    for j in range(0,9):
        val = val + confusion[i][j] * (((j+1) - (i+1))^2) 
print(val / sum(sum(confusion)))

1.054054054054054


## 3. G Modeling: LGBM, RandomForest, AutoML

### 3-1) LGBM

In [136]:
## 전체 데이터를 8 : 1 : 1 로 TRAIN, VALID, TEST로 나눠야됨

train, validate, test = np.split(G_train.sample(frac=1, random_state=48), [int(.8*len(G_train)), int(.9*len(G_train))])

In [137]:
x_train = train.drop(['G_GRADE'], axis = 1)
y_train = train['G_GRADE']
x_validate = validate.drop(['G_GRADE'], axis = 1)
y_validate = validate['G_GRADE']
x_test = test.drop(['G_GRADE'], axis = 1)
y_test = test['G_GRADE']

In [138]:
# LGBM의 하이퍼 파라미터의 범위를 dictionary 형태로 지정

lgbm_parameter_bounds = {'n_estimators' : (100,400), 
                         'num_leaves' : (16,40), 
                         'learning_rate' : (0.05 , 0.3), 
                         'early_stopping_rounds' : (100 , 100),
                        }

def lgbm_bo(n_estimators, num_leaves, learning_rate, early_stopping_rounds):
    lgbm_params = {'n_estimators' : int(round(n_estimators)),'num_leaves' : int(round(num_leaves)), 'learning_rate' : float(learning_rate), 'early_stopping_rounds' : int(round(early_stopping_rounds))}
    lgbm = LGBMClassifier(**lgbm_params, application = 'multiclass' , random_state = 48)
    evals = [(x_validate, y_validate)]
    lgbm.fit(x_train,y_train, eval_metric='multi_logloss', eval_set=evals)
    F1_score = f1_score(y_test, lgbm.predict(x_test), average = 'macro')

    return F1_score



# 이제 Bayesian Optimization을 사용할 준비가 끝났습니다.

# "BO_lgbm"라는 변수에 Bayesian Optmization을 저장해보세요

BO_lgbm = BayesianOptimization(f = lgbm_bo, pbounds = lgbm_parameter_bounds, random_state = 48)



# Bayesian Optimization을 실행해보세요

BO_lgbm.maximize(init_points = 5, n_iter = 30)

|   iter    |  target   | early_... | learni... | n_esti... | num_le... |
-------------------------------------------------------------------------
[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
[1]	valid_0's multi_logloss: 2.03301
[2]	valid_0's multi_logloss: 1.93708
[3]	valid_0's multi_logloss: 1.88914
[4]	valid_0's multi_logloss: 1.86031
[5]	valid_0's multi_logloss: 1.85056
[6]	valid_0's multi_logloss: 1.82045
[7]	valid_0's multi_logloss: 1.82372
[8]	valid_0's multi_logloss: 1.82009
[9]	valid_0's multi_logloss: 1.81332
[10]	valid_0's multi_logloss: 1.81309
[11]	valid_0's multi_logloss: 1.79818
[12]	valid_0's multi_logloss: 1.78558
[13]	valid_0's multi_logloss: 1.78439
[14]	valid_0's multi_logloss: 1.77556
[15]	valid_0's multi_logloss: 1.77508
[16]	valid_0's multi_logloss: 1.75572
[17]	valid_0's multi_logloss: 1.74798
[18]	valid_0's multi_logloss: 1.73493
[19]	valid_0's multi_logloss: 1.73512
[2

/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.p

[19]	valid_0's multi_logloss: 1.77915
[20]	valid_0's multi_logloss: 1.77629
[21]	valid_0's multi_logloss: 1.77405
[22]	valid_0's multi_logloss: 1.77667
[23]	valid_0's multi_logloss: 1.77499
[24]	valid_0's multi_logloss: 1.76603
[25]	valid_0's multi_logloss: 1.76139
[26]	valid_0's multi_logloss: 1.76301
[27]	valid_0's multi_logloss: 1.75871
[28]	valid_0's multi_logloss: 1.7545
[29]	valid_0's multi_logloss: 1.74864
[30]	valid_0's multi_logloss: 1.74543
[31]	valid_0's multi_logloss: 1.74332
[32]	valid_0's multi_logloss: 1.7397
[33]	valid_0's multi_logloss: 1.73707
[34]	valid_0's multi_logloss: 1.73765
[35]	valid_0's multi_logloss: 1.73683
[36]	valid_0's multi_logloss: 1.7313
[37]	valid_0's multi_logloss: 1.73688
[38]	valid_0's multi_logloss: 1.73909
[39]	valid_0's multi_logloss: 1.74051
[40]	valid_0's multi_logloss: 1.74206
[41]	valid_0's multi_logloss: 1.73872
[42]	valid_0's multi_logloss: 1.74153
[43]	valid_0's multi_logloss: 1.74451
[44]	valid_0's multi_logloss: 1.74663
[45]	valid_0's 

/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.p

[1]	valid_0's multi_logloss: 2.04608
[2]	valid_0's multi_logloss: 1.93577
[3]	valid_0's multi_logloss: 1.88978
[4]	valid_0's multi_logloss: 1.87281
[5]	valid_0's multi_logloss: 1.83544
[6]	valid_0's multi_logloss: 1.81837
[7]	valid_0's multi_logloss: 1.81625
[8]	valid_0's multi_logloss: 1.80182
[9]	valid_0's multi_logloss: 1.79517
[10]	valid_0's multi_logloss: 1.79145
[11]	valid_0's multi_logloss: 1.78991
[12]	valid_0's multi_logloss: 1.79699
[13]	valid_0's multi_logloss: 1.78866
[14]	valid_0's multi_logloss: 1.77661
[15]	valid_0's multi_logloss: 1.77127
[16]	valid_0's multi_logloss: 1.7625
[17]	valid_0's multi_logloss: 1.76537
[18]	valid_0's multi_logloss: 1.76222
[19]	valid_0's multi_logloss: 1.76429
[20]	valid_0's multi_logloss: 1.76348
[21]	valid_0's multi_logloss: 1.76275
[22]	valid_0's multi_logloss: 1.76828
[23]	valid_0's multi_logloss: 1.76636
[24]	valid_0's multi_logloss: 1.77127
[25]	valid_0's multi_logloss: 1.77542
[26]	valid_0's multi_logloss: 1.77204
[27]	valid_0's multi_l

...

[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
[1]	valid_0's multi_logloss: 2.142
[2]	valid_0's multi_logloss: 2.09588
[3]	valid_0's multi_logloss: 2.05225
[4]	valid_0's multi_logloss: 2.01759
[5]	valid_0's multi_logloss: 1.99148
[6]	valid_0's multi_logloss: 1.96987
[7]	valid_0's multi_logloss: 1.95034
[8]	valid_0's multi_logloss: 1.92751
[9]	valid_0's multi_logloss: 1.91099
[10]	valid_0's multi_logloss: 1.89636
[11]	valid_0's multi_logloss: 1.88158
[12]	valid_0's multi_logloss: 1.87402
[13]	valid_0's multi_logloss: 1.86864
[14]	valid_0's multi_logloss: 1.85767
[15]	valid_0's multi_logloss: 1.85149
[16]	valid_0's multi_logloss: 1.84755
[17]	valid_0's multi_logloss: 1.84727
[18]	valid_0's multi_logloss: 1.84368
[19]	valid_0's multi_logloss: 1.83342
[20]	valid_0's multi_logloss: 1.82736
[21]	valid_0's multi_logloss: 1.82114
[22]	valid_0's multi_logloss: 1.81827
[23]	valid_0's multi_logloss: 1.81554


/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.p

[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
[1]	valid_0's multi_logloss: 2.08522
[2]	valid_0's multi_logloss: 2.01222
[3]	valid_0's multi_logloss: 1.97081
[4]	valid_0's multi_logloss: 1.93696
[5]	valid_0's multi_logloss: 1.90583
[6]	valid_0's multi_logloss: 1.88878
[7]	valid_0's multi_logloss: 1.86949
[8]	valid_0's multi_logloss: 1.84277
[9]	valid_0's multi_logloss: 1.82688
[10]	valid_0's multi_logloss: 1.81867
[11]	valid_0's multi_logloss: 1.8037
[12]	valid_0's multi_logloss: 1.79478
[13]	valid_0's multi_logloss: 1.78559
[14]	valid_0's multi_logloss: 1.77099
[15]	valid_0's multi_logloss: 1.7742
[16]	valid_0's multi_logloss: 1.77176
[17]	valid_0's multi_logloss: 1.77671
[18]	valid_0's multi_logloss: 1.78068
[19]	valid_0's multi_logloss: 1.77706
[20]	valid_0's multi_logloss: 1.77874
[21]	valid_0's multi_logloss: 1.77734
[22]	valid_0's multi_logloss: 1.76292
[23]	valid_0's multi_logloss: 1.76418


/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.p

[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
[1]	valid_0's multi_logloss: 2.06661
[2]	valid_0's multi_logloss: 1.98073
[3]	valid_0's multi_logloss: 1.95636
[4]	valid_0's multi_logloss: 1.92184
[5]	valid_0's multi_logloss: 1.90308
[6]	valid_0's multi_logloss: 1.87335
[7]	valid_0's multi_logloss: 1.83732
[8]	valid_0's multi_logloss: 1.82924
[9]	valid_0's multi_logloss: 1.82764
[10]	valid_0's multi_logloss: 1.82031
[11]	valid_0's multi_logloss: 1.80424
[12]	valid_0's multi_logloss: 1.78233
[13]	valid_0's multi_logloss: 1.77628
[14]	valid_0's multi_logloss: 1.77699
[15]	valid_0's multi_logloss: 1.7704
[16]	valid_0's multi_logloss: 1.76881
[17]	valid_0's multi_logloss: 1.76891
[18]	valid_0's multi_logloss: 1.75452
[19]	valid_0's multi_logloss: 1.75144
[20]	valid_0's multi_logloss: 1.74951
[21]	valid_0's multi_logloss: 1.74554
[22]	valid_0's multi_logloss: 1.74286
[23]	valid_0's multi_logloss: 1.74274

In [139]:
max_params = BO_lgbm.max['params']
max_params['learning_rate'] = float(max_params['learning_rate'])
max_params['n_estimators'] = int(max_params['n_estimators'])
max_params['num_leaves'] = int(max_params['num_leaves'])
print(max_params)

{'early_stopping_rounds': 100.0, 'learning_rate': 0.18915469323456902, 'n_estimators': 156, 'num_leaves': 25}


In [140]:
G_lgbm_tune =LGBMClassifier(n_estimators = 156 , learning_rate = 0.189, early_stopping_rounds = 100, num_leaves = 25)
evals = [(x_validate, y_validate)]
G_lgbm_tune.fit(x_train,y_train, eval_metric='multi_logloss', eval_set=evals)

[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
[1]	valid_0's multi_logloss: 2.07228
[2]	valid_0's multi_logloss: 1.99415
[3]	valid_0's multi_logloss: 1.95473
[4]	valid_0's multi_logloss: 1.91073
[5]	valid_0's multi_logloss: 1.87776
[6]	valid_0's multi_logloss: 1.8502
[7]	valid_0's multi_logloss: 1.83969
[8]	valid_0's multi_logloss: 1.83704
[9]	valid_0's multi_logloss: 1.81166
[10]	valid_0's multi_logloss: 1.79813
[11]	valid_0's multi_logloss: 1.79543
[12]	valid_0's multi_logloss: 1.79713
[13]	valid_0's multi_logloss: 1.80032
[14]	valid_0's multi_logloss: 1.79452
[15]	valid_0's multi_logloss: 1.78573
[16]	valid_0's multi_logloss: 1.78349
[17]	valid_0's multi_logloss: 1.77707
[18]	valid_0's multi_logloss: 1.77107
[19]	valid_0's multi_logloss: 1.76631
[20]	valid_0's multi_logloss: 1.77089
[21]	valid_0's multi_logloss: 1.7642
[22]	valid_0's multi_logloss: 1.75527
[23]	valid_0's multi_logloss: 1.7546
[

LGBMClassifier(early_stopping_rounds=100, learning_rate=0.189, n_estimators=156,
               num_leaves=25)

In [141]:
x_test_new = pd.concat([x_validate , x_test])
y_test_new = pd.concat([y_validate , y_test])

In [142]:
pred = G_lgbm_tune.predict(x_test_new)

In [143]:
get_clf_eval(y_test_new, pred)

오차행렬:
 [[15  4  3  1  0  0  0  1  0]
 [ 2 15  0  1  2  1  1  1  0]
 [ 2  3 12  2  2  0  1  2  4]
 [ 0  1  4 11  0  0  0  6  2]
 [ 0  0  0  2  8  7  0  4  5]
 [ 0  1  1  2  1 13  1  3  1]
 [ 0  1  1  3  1  2 12  6  4]
 [ 0  1  1  4  3  2  1 10  1]
 [ 0  2  0  0  2  3  3  4  7]]

정확도: 0.4640
정밀도: 0.4858
재현율: 0.4672
F1: 0.4673


In [144]:
confusion = confusion_matrix(y_test_new, pred)

val = 0
for i in range(0,9):
    for j in range(0,9):
        val = val + confusion[i][j] * (((j+1) - (i+1))^2) 
print(val / sum(sum(confusion)))

1.1531531531531531


### 3-2) RandomForest

In [145]:
## 전체 데이터를 8 : 2 로 TRAIN, TEST로 나눠야됨

train, test = np.split(G_train.sample(frac=1, random_state=48), [int(.8*len(G_train))])

In [146]:
x_train = train.drop(['G_GRADE'], axis = 1)
y_train = train['G_GRADE']
x_test = test.drop(['G_GRADE'], axis = 1)
y_test = test['G_GRADE']

In [147]:
# 랜덤 포레스트의 하이퍼 파라미터의 범위를 dictionary 형태로 지정

RF_parameter_bounds = {'n_estimators' : (5,15), 'max_depth' : (1,50), 'min_samples_leaf' : (1, 10), 'min_samples_split' : (2,10)}

def RF_bo(n_estimators, max_depth, min_samples_leaf, min_samples_split):
    RF_params = {'n_estimators' : int(round(n_estimators)),
                 'max_depth' : int(round(max_depth)), 
                 'min_samples_leaf' : int(round(min_samples_leaf)),
                 'min_samples_split' : int(round(min_samples_split))}
    RF =  RandomForestClassifier(**RF_params, random_state = 48)
    RF.fit(x_train,y_train)
    F1_score = f1_score(y_test, RF.predict(x_test), average = 'macro')

    return F1_score



# 이제 Bayesian Optimization을 사용할 준비가 끝났습니다.

# "BO_lgbm"라는 변수에 Bayesian Optmization을 저장해보세요

BO_RF = BayesianOptimization(f = RF_bo, pbounds = RF_parameter_bounds, random_state = 48)



# Bayesian Optimization을 실행해보세요

BO_RF.maximize(init_points = 5, n_iter = 50)

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        |  0.1263   |  1.857    |  9.024    |  4.279    |  7.99     |
|  2        |  0.3259   |  39.81    |  3.92     |  8.918    |  9.475    |
|  3        |  0.329    |  27.86    |  4.215    |  2.898    |  6.419    |
|  4        |  0.2487   |  22.8     |  7.588    |  5.681    |  10.93    |
|  5        |  0.298    |  17.5     |  5.09     |  3.497    |  9.088    |
|  6        |  0.326    |  49.94    |  3.114    |  5.928    |  9.33     |
|  7        |  0.2643   |  38.81    |  8.307    |  2.0      |  5.0      |
|  8        |  0.1955   |  5.224    |  7.28     |  9.798    |  7.824    |
|  9        |  0.4626   |  42.88    |  1.0      |  2.645    |  15.0     |
|  10       |  0.4947   |  36.43    |  1.0      |  2.0      |  15.0     |
|  11       |  0.4947   |  29.62    |  1.0      |  2.0      |  15.0     |
|  12       |  0.3791   |  31.9     | 

In [148]:
max_params = BO_RF.max['params']
max_params['n_estimators'] = int(max_params['n_estimators'])
max_params['max_depth'] = int(max_params['max_depth'])
max_params['min_samples_leaf'] = int(max_params['min_samples_leaf'])
max_params['min_samples_split'] = int(max_params['min_samples_split'])
print(max_params)

{'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 15}


In [149]:
G_RF_tune =  RandomForestClassifier(max_depth = 20, min_samples_leaf = 1, n_estimators = 15, min_samples_split = 2 ,random_state = 48)
G_RF_tune.fit(x_train,y_train)

pred = G_RF_tune.predict(x_test)

In [150]:
get_clf_eval(y_test, pred)

오차행렬:
 [[18  5  0  0  0  0  0  1  0]
 [ 2 14  0  1  4  0  1  1  0]
 [ 4  0 11  1  4  2  1  3  2]
 [ 0  0  4 11  2  0  0  5  2]
 [ 0  0  1  2 11  5  0  3  4]
 [ 0  1  1  2  1 14  1  2  1]
 [ 0  0  1  4  2  3 13  2  5]
 [ 0  1  1  3  3  3  1 10  1]
 [ 0  1  0  0  3  2  3  3  9]]

정확도: 0.5000
정밀도: 0.5146
재현율: 0.5043
F1: 0.5030


In [151]:
confusion = confusion_matrix(y_test, pred)

val = 0
for i in range(0,9):
    for j in range(0,9):
        val = val + confusion[i][j] * (((j+1) - (i+1))^2) 
print(val / sum(sum(confusion)))

1.117117117117117


### 3-3) AutoML

In [152]:
from google.cloud import aiplatform
import json
G_endpoint = aiplatform.Endpoint(
    endpoint_name=f"projects/{PROJECT_ID}/locations/asia-northeast3/endpoints/1769597994207805440"
)
x_test_automl=x_test.astype(str)
response = G_endpoint.predict(json.loads(x_test_automl.to_json(orient='records')))
pred_ml=[]
for row in response[0]:
    scores=row["scores"]
    for i,value in enumerate(scores):
        if value==max(row["scores"]):
            #print(row.index[i])
            pred_ml.append(int(row["classes"][i].replace("'","")))
get_clf_eval(y_test, pred_ml)

오차행렬:
 [[16  5  2  0  0  0  0  1  0]
 [ 5  9  0  2  3  0  2  2  0]
 [ 3  2  5  5  3  1  4  4  1]
 [ 0  5  3  5  1  1  3  4  2]
 [ 0  2  0  1  7  5  4  7  0]
 [ 1  1  0  5  4  7  2  2  1]
 [ 0  1  0  3  3  3 14  6  0]
 [ 0  0  1  4  2  2  3 11  0]
 [ 0  1  1  1  4  2  4  5  3]]

정확도: 0.3468
정밀도: 0.3630
재현율: 0.3451
F1: 0.3366


In [153]:
confusion = confusion_matrix(y_test, pred_ml)

val = 0
for i in range(0,9):
    for j in range(0,9):
        val = val + confusion[i][j] * (((j+1) - (i+1))^2) 
print(val / sum(sum(confusion)))

0.5765765765765766


## 4. ESG Modeling: LGBM, RandomForest, AutoML

### 4-1) LGBM

In [154]:
## 전체 데이터를 8 : 1 : 1 로 TRAIN, VALID, TEST로 나눠야됨

train, validate, test = np.split(ESG_train.sample(frac=1, random_state=48), [int(.8*len(ESG_train)), int(.9*len(ESG_train))])

In [155]:
x_train = train.drop(['ESG_GRADE'], axis = 1)
y_train = train['ESG_GRADE']
x_validate = validate.drop(['ESG_GRADE'], axis = 1)
y_validate = validate['ESG_GRADE']
x_test = test.drop(['ESG_GRADE'], axis = 1)
y_test = test['ESG_GRADE']

In [156]:
# LGBM의 하이퍼 파라미터의 범위를 dictionary 형태로 지정

lgbm_parameter_bounds = {'n_estimators' : (100,400), 
                         'num_leaves' : (16,40), 
                         'learning_rate' : (0.05 , 0.3), 
                         'early_stopping_rounds' : (100 , 100),
                        }

def lgbm_bo(n_estimators, num_leaves, learning_rate, early_stopping_rounds):
    lgbm_params = {'n_estimators' : int(round(n_estimators)),'num_leaves' : int(round(num_leaves)), 'learning_rate' : float(learning_rate), 'early_stopping_rounds' : int(round(early_stopping_rounds))}
    lgbm = LGBMClassifier(**lgbm_params , application = 'multiclass' , random_state = 48)
    evals = [(x_validate, y_validate)]
    lgbm.fit(x_train,y_train, eval_metric='multi_logloss', eval_set=evals)
    F1_score = f1_score(y_test, lgbm.predict(x_test), average = 'macro')

    return F1_score



# 이제 Bayesian Optimization을 사용할 준비가 끝났습니다.

# "BO_lgbm"라는 변수에 Bayesian Optmization을 저장해보세요

BO_lgbm = BayesianOptimization(f = lgbm_bo, pbounds = lgbm_parameter_bounds, random_state = 48)



# Bayesian Optimization을 실행해보세요

BO_lgbm.maximize(init_points = 5, n_iter = 30)

|   iter    |  target   | early_... | learni... | n_esti... | num_le... |
-------------------------------------------------------------------------
[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
[1]	valid_0's multi_logloss: 1.71385
[2]	valid_0's multi_logloss: 1.51258
[3]	valid_0's multi_logloss: 1.38843
[4]	valid_0's multi_logloss: 1.31065
[5]	valid_0's multi_logloss: 1.24266
[6]	valid_0's multi_logloss: 1.19637
[7]	valid_0's multi_logloss: 1.14488
[8]	valid_0's multi_logloss: 1.11638
[9]	valid_0's multi_logloss: 1.07522
[10]	valid_0's multi_logloss: 1.03954
[11]	valid_0's multi_logloss: 1.01077
[12]	valid_0's multi_logloss: 1.00558
[13]	valid_0's multi_logloss: 0.966523
[14]	valid_0's multi_logloss: 0.952174
[15]	valid_0's multi_logloss: 0.932079
[16]	valid_0's multi_logloss: 0.921728
[17]	valid_0's multi_logloss: 0.910092
[18]	valid_0's multi_logloss: 0.90167
[19]	valid_0's multi_logloss: 0.903

/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.p

[7]	valid_0's multi_logloss: 1.39744
[8]	valid_0's multi_logloss: 1.34677
[9]	valid_0's multi_logloss: 1.30345
[10]	valid_0's multi_logloss: 1.27065
[11]	valid_0's multi_logloss: 1.23698
[12]	valid_0's multi_logloss: 1.20557
[13]	valid_0's multi_logloss: 1.18335
[14]	valid_0's multi_logloss: 1.16028
[15]	valid_0's multi_logloss: 1.1357
[16]	valid_0's multi_logloss: 1.11928
[17]	valid_0's multi_logloss: 1.09357
[18]	valid_0's multi_logloss: 1.06964
[19]	valid_0's multi_logloss: 1.04764
[20]	valid_0's multi_logloss: 1.03269
[21]	valid_0's multi_logloss: 1.02202
[22]	valid_0's multi_logloss: 1.00398
[23]	valid_0's multi_logloss: 0.988178
[24]	valid_0's multi_logloss: 0.964985
[25]	valid_0's multi_logloss: 0.951362
[26]	valid_0's multi_logloss: 0.939943
[27]	valid_0's multi_logloss: 0.932241
[28]	valid_0's multi_logloss: 0.919884
[29]	valid_0's multi_logloss: 0.909639
[30]	valid_0's multi_logloss: 0.899232
[31]	valid_0's multi_logloss: 0.892417
[32]	valid_0's multi_logloss: 0.888339
[33]	v

/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.p

[18]	valid_0's multi_logloss: 0.841612
[19]	valid_0's multi_logloss: 0.839133
[20]	valid_0's multi_logloss: 0.841352
[21]	valid_0's multi_logloss: 0.835274
[22]	valid_0's multi_logloss: 0.841917
[23]	valid_0's multi_logloss: 0.844497
[24]	valid_0's multi_logloss: 0.853455
[25]	valid_0's multi_logloss: 0.848231
[26]	valid_0's multi_logloss: 0.851781
[27]	valid_0's multi_logloss: 0.854438
[28]	valid_0's multi_logloss: 0.861118
[29]	valid_0's multi_logloss: 0.851954
[30]	valid_0's multi_logloss: 0.864623
[31]	valid_0's multi_logloss: 0.856831
[32]	valid_0's multi_logloss: 0.867399
[33]	valid_0's multi_logloss: 0.878054
[34]	valid_0's multi_logloss: 0.885406
[35]	valid_0's multi_logloss: 0.888241
[36]	valid_0's multi_logloss: 0.899272
[37]	valid_0's multi_logloss: 0.905886
[38]	valid_0's multi_logloss: 0.917036
[39]	valid_0's multi_logloss: 0.917769
[40]	valid_0's multi_logloss: 0.929202
[41]	valid_0's multi_logloss: 0.936113
[42]	valid_0's multi_logloss: 0.932277
[43]	valid_0's multi_logl

...

[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
[1]	valid_0's multi_logloss: 1.76594
[2]	valid_0's multi_logloss: 1.5871
[3]	valid_0's multi_logloss: 1.47458
[4]	valid_0's multi_logloss: 1.36208
[5]	valid_0's multi_logloss: 1.27823
[6]	valid_0's multi_logloss: 1.20794
[7]	valid_0's multi_logloss: 1.15335
[8]	valid_0's multi_logloss: 1.11206
[9]	valid_0's multi_logloss: 1.06088
[10]	valid_0's multi_logloss: 1.03071
[11]	valid_0's multi_logloss: 0.992273
[12]	valid_0's multi_logloss: 0.961896
[13]	valid_0's multi_logloss: 0.95192
[14]	valid_0's multi_logloss: 0.927117
[15]	valid_0's multi_logloss: 0.926622
[16]	valid_0's multi_logloss: 0.907437
[17]	valid_0's multi_logloss: 0.891156
[18]	valid_0's multi_logloss: 0.87634
[19]	valid_0's multi_logloss: 0.856929
[20]	valid_0's multi_logloss: 0.854033
[21]	valid_0's multi_logloss: 0.848693
[22]	valid_0's multi_logloss: 0.847674
[23]	valid_0's multi_loglos

/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.p

[1]	valid_0's multi_logloss: 1.83925
[2]	valid_0's multi_logloss: 1.671
[3]	valid_0's multi_logloss: 1.54372
[4]	valid_0's multi_logloss: 1.4649
[5]	valid_0's multi_logloss: 1.37794
[6]	valid_0's multi_logloss: 1.32429
[7]	valid_0's multi_logloss: 1.26483
[8]	valid_0's multi_logloss: 1.223
[9]	valid_0's multi_logloss: 1.17153
[10]	valid_0's multi_logloss: 1.13198
[11]	valid_0's multi_logloss: 1.09698
[12]	valid_0's multi_logloss: 1.06635
[13]	valid_0's multi_logloss: 1.0454
[14]	valid_0's multi_logloss: 1.02377
[15]	valid_0's multi_logloss: 0.993599
[16]	valid_0's multi_logloss: 0.972221
[17]	valid_0's multi_logloss: 0.959817
[18]	valid_0's multi_logloss: 0.934696
[19]	valid_0's multi_logloss: 0.919504
[20]	valid_0's multi_logloss: 0.908702
[21]	valid_0's multi_logloss: 0.901502
[22]	valid_0's multi_logloss: 0.886396
[23]	valid_0's multi_logloss: 0.868709
[24]	valid_0's multi_logloss: 0.858711
[25]	valid_0's multi_logloss: 0.857696
[26]	valid_0's multi_logloss: 0.850804
[27]	valid_0's 

/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/opt/conda/lib/python3.7/site-packages/scipy/optimize/_numdiff.p

[1]	valid_0's multi_logloss: 1.73294
[2]	valid_0's multi_logloss: 1.54179
[3]	valid_0's multi_logloss: 1.39407
[4]	valid_0's multi_logloss: 1.29436
[5]	valid_0's multi_logloss: 1.23578
[6]	valid_0's multi_logloss: 1.17326
[7]	valid_0's multi_logloss: 1.12025
[8]	valid_0's multi_logloss: 1.08121
[9]	valid_0's multi_logloss: 1.03891
[10]	valid_0's multi_logloss: 0.988731
[11]	valid_0's multi_logloss: 0.975473
[12]	valid_0's multi_logloss: 0.956322
[13]	valid_0's multi_logloss: 0.93051
[14]	valid_0's multi_logloss: 0.905718
[15]	valid_0's multi_logloss: 0.887149
[16]	valid_0's multi_logloss: 0.872446
[17]	valid_0's multi_logloss: 0.874354
[18]	valid_0's multi_logloss: 0.852898
[19]	valid_0's multi_logloss: 0.838353
[20]	valid_0's multi_logloss: 0.836845
[21]	valid_0's multi_logloss: 0.833958
[22]	valid_0's multi_logloss: 0.830465
[23]	valid_0's multi_logloss: 0.840392
[24]	valid_0's multi_logloss: 0.842308
[25]	valid_0's multi_logloss: 0.842998
[26]	valid_0's multi_logloss: 0.834796
[27]	

In [157]:
max_params = BO_lgbm.max['params']
max_params['learning_rate'] = float(max_params['learning_rate'])
max_params['n_estimators'] = int(max_params['n_estimators'])
max_params['num_leaves'] = int(max_params['num_leaves'])
print(max_params)

{'early_stopping_rounds': 100.0, 'learning_rate': 0.17174091183439083, 'n_estimators': 265, 'num_leaves': 31}


In [158]:
ESG_lgbm_tune =LGBMClassifier(n_estimators = 265 , learning_rate = 0.172, early_stopping_rounds = 100, num_leaves = 31)
evals = [(x_validate, y_validate)]
ESG_lgbm_tune.fit(x_train,y_train, eval_metric='multi_logloss', eval_set=evals)

[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
[1]	valid_0's multi_logloss: 1.83103
[2]	valid_0's multi_logloss: 1.65727
[3]	valid_0's multi_logloss: 1.53211
[4]	valid_0's multi_logloss: 1.43933
[5]	valid_0's multi_logloss: 1.35153
[6]	valid_0's multi_logloss: 1.28927
[7]	valid_0's multi_logloss: 1.23334
[8]	valid_0's multi_logloss: 1.19322
[9]	valid_0's multi_logloss: 1.15112
[10]	valid_0's multi_logloss: 1.11251
[11]	valid_0's multi_logloss: 1.08968
[12]	valid_0's multi_logloss: 1.05586
[13]	valid_0's multi_logloss: 1.03251
[14]	valid_0's multi_logloss: 1.00386
[15]	valid_0's multi_logloss: 0.979002
[16]	valid_0's multi_logloss: 0.957569
[17]	valid_0's multi_logloss: 0.936876
[18]	valid_0's multi_logloss: 0.914367
[19]	valid_0's multi_logloss: 0.900521
[20]	valid_0's multi_logloss: 0.887299
[21]	valid_0's multi_logloss: 0.876802
[22]	valid_0's multi_logloss: 0.870394
[23]	valid_0's multi_logloss

LGBMClassifier(early_stopping_rounds=100, learning_rate=0.172, n_estimators=265)

In [159]:
x_test_new = pd.concat([x_validate , x_test])
y_test_new = pd.concat([y_validate , y_test])

In [160]:
pred = ESG_lgbm_tune.predict(x_test_new)

In [161]:
get_clf_eval(y_test_new, pred)

오차행렬:
 [[ 5  0  0  0  0  0  0  0  0]
 [ 0  7  3  0  2  0  1  0  0]
 [ 0  0 12  0  0  0  0  0  0]
 [ 0  0  0 25  0  0  0  0  0]
 [ 0  0  0  0 11  0  0  3  3]
 [ 0  0  0  0  1 20  2  0  0]
 [ 0  1  0  2  0  2 16 10  0]
 [ 0  0  0  2  1  1  8 30  8]
 [ 0  0  0  1  0  0  4  8 28]]

정확도: 0.7097
정밀도: 0.7704
재현율: 0.7616
F1: 0.7591


In [162]:
confusion = confusion_matrix(y_test_new, pred)

val = 0
for i in range(0,9):
    for j in range(0,9):
        val = val + confusion[i][j] * (((j+1) - (i+1))^2) 
print(val / sum(sum(confusion)))

1.423963133640553


### 4-2) RandomForest

In [163]:
## 전체 데이터를 8 : 2 로 TRAIN, TEST로 나눠야됨

train, test = np.split(ESG_train.sample(frac=1, random_state=48), [int(.8*len(ESG_train))])

In [164]:
x_train = train.drop(['ESG_GRADE'], axis = 1)
y_train = train['ESG_GRADE']
x_test = test.drop(['ESG_GRADE'], axis = 1)
y_test = test['ESG_GRADE']

In [165]:
# 랜덤 포레스트의 하이퍼 파라미터의 범위를 dictionary 형태로 지정

RF_parameter_bounds = {'n_estimators' : (5,15), 'max_depth' : (1,50), 'min_samples_leaf' : (1, 10), 'min_samples_split' : (2,10)}

def RF_bo(n_estimators, max_depth, min_samples_leaf, min_samples_split):
    RF_params = {'n_estimators' : int(round(n_estimators)),
                 'max_depth' : int(round(max_depth)), 
                 'min_samples_leaf' : int(round(min_samples_leaf)),
                 'min_samples_split' : int(round(min_samples_split))}
    RF =  RandomForestClassifier(**RF_params, random_state = 48)
    RF.fit(x_train,y_train)
    F1_score = f1_score(y_test, RF.predict(x_test), average = 'macro')

    return F1_score



# 이제 Bayesian Optimization을 사용할 준비가 끝났습니다.

# "BO_lgbm"라는 변수에 Bayesian Optmization을 저장해보세요

BO_RF = BayesianOptimization(f = RF_bo, pbounds = RF_parameter_bounds, random_state = 48)



# Bayesian Optimization을 실행해보세요

BO_RF.maximize(init_points = 5, n_iter = 50)

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        |  0.2356   |  1.857    |  9.024    |  4.279    |  7.99     |
|  2        |  0.57     |  39.81    |  3.92     |  8.918    |  9.475    |
|  3        |  0.6183   |  27.86    |  4.215    |  2.898    |  6.419    |
|  4        |  0.494    |  22.8     |  7.588    |  5.681    |  10.93    |
|  5        |  0.5682   |  17.5     |  5.09     |  3.497    |  9.088    |
|  6        |  0.6218   |  49.94    |  3.114    |  5.928    |  9.33     |
|  7        |  0.4022   |  45.03    |  10.0     |  2.0      |  5.0      |
|  8        |  0.5249   |  5.224    |  7.28     |  9.798    |  7.824    |
|  9        |  0.5249   |  5.248    |  7.129    |  9.536    |  8.181    |
|  10       |  0.6484   |  46.4     |  1.139    |  8.769    |  10.88    |
|  11       |  0.5877   |  50.0     |  3.457    |  9.804    |  15.0     |
|  12       |  0.7426   |  45.83    | 

In [166]:
max_params = BO_RF.max['params']
max_params['n_estimators'] = int(max_params['n_estimators'])
max_params['max_depth'] = int(max_params['max_depth'])
max_params['min_samples_leaf'] = int(max_params['min_samples_leaf'])
max_params['min_samples_split'] = int(max_params['min_samples_split'])
print(max_params)

{'max_depth': 13, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 15}


In [167]:
ESG_RF_tune =  RandomForestClassifier(max_depth = 13, min_samples_leaf = 1, n_estimators = 15, min_samples_split = 2 ,random_state = 48)
ESG_RF_tune.fit(x_train,y_train)

pred = ESG_RF_tune.predict(x_test)

In [168]:
get_clf_eval(y_test, pred)

오차행렬:
 [[ 5  0  0  0  0  0  0  0  0]
 [ 0  7  3  0  2  0  1  0  0]
 [ 0  0 11  0  0  1  0  0  0]
 [ 0  0  0 24  0  0  1  0  0]
 [ 0  0  0  0 10  1  0  6  0]
 [ 0  0  0  1  0 20  2  0  0]
 [ 0  1  0  0  0  2 18  8  2]
 [ 0  0  0  1  3  1 11 24 10]
 [ 0  0  0  0  0  0  1 12 28]]

정확도: 0.6774
정밀도: 0.7511
재현율: 0.7352
F1: 0.7375


In [169]:
confusion = confusion_matrix(y_test, pred)

val = 0
for i in range(0,9):
    for j in range(0,9):
        val = val + confusion[i][j] * (((j+1) - (i+1))^2) 
print(val / sum(sum(confusion)))

1.3087557603686637


### 4-3) AutoML

In [170]:
from google.cloud import aiplatform
import json
endpoint = aiplatform.Endpoint(
    endpoint_name=f"projects/{PROJECT_ID}/locations/asia-northeast3/endpoints/1745954096164110336"
)
x_test_automl=x_test.astype(str)
response = endpoint.predict(json.loads(x_test_automl.to_json(orient='records')))
pred_ml=[]
for row in response[0]:
    scores=row["scores"]
    for i,value in enumerate(scores):
        if value==max(row["scores"]):
            #print(row.index[i])
            pred_ml.append(int(row["classes"][i].replace("'","")))
get_clf_eval(y_test, pred_ml)

오차행렬:
 [[ 5  0  0  0  0  0  0  0  0]
 [ 0 10  0  0  3  0  0  0  0]
 [ 0  0 11  0  0  0  0  1  0]
 [ 0  1  0 17  1  1  2  2  1]
 [ 0  0  0  0  8  1  3  2  3]
 [ 0  0  0  0  1 19  1  2  0]
 [ 0  1  0  0  0  1 15 12  2]
 [ 0  0  0  2  1  0  5 32 10]
 [ 0  0  0  1  0  0  0  7 33]]

정확도: 0.6912
정밀도: 0.7689
재현율: 0.7324
F1: 0.7472


In [171]:
confusion = confusion_matrix(y_test, pred_ml)

val = 0
for i in range(0,9):
    for j in range(0,9):
        val = val + confusion[i][j] * (((j+1) - (i+1))^2) 
print(val / sum(sum(confusion)))

1.663594470046083


## 5. Final model & prediction

In [172]:
## TRAIN 데이터 불러오기
E_train=pd.read_csv("data/pipeline3/modeling_data/E_train.csv")
S_train=pd.read_csv("data/pipeline3/modeling_data/S_train.csv")
G_train=pd.read_csv("data/pipeline3/modeling_data/G_train.csv")
ESG_train=pd.read_csv("data/pipeline3/modeling_data/ESG_train.csv")

## test 데이터 불러오기
E_test=pd.read_csv("data/pipeline3/modeling_data/E_test.csv")
S_test=pd.read_csv("data/pipeline3/modeling_data/S_test.csv")
G_test=pd.read_csv("data/pipeline3/modeling_data/G_test.csv")
ESG_test=pd.read_csv("data/pipeline3/modeling_data/ESG_test.csv")

In [173]:
E_pred = E_RF_tune.predict(E_test.drop(["code"],axis=1))
S_pred = S_RF_tune.predict(S_test.drop(["code"],axis=1))
G_pred = G_RF_tune.predict(G_test.drop(["code"],axis=1))
ESG_pred = ESG_RF_tune.predict(ESG_test.drop(["code"],axis=1))

In [174]:
E_test["E_GRADE"]=E_pred
S_test["S_GRADE"]=S_pred
G_test["G_GRADE"]=G_pred
ESG_test["ESG_GRADE"]=ESG_pred

In [175]:
# 정답채워넣기
final_answer=pd.read_csv("data/final_esg/esg_test_data.csv")

In [176]:
final_answer=pd.merge(final_answer[["종목코드","종목명","시장"]],E_test[["code","E_GRADE"]],right_on="code",left_on="종목코드",how="left").drop(["code"],axis=1)
final_answer=pd.merge(final_answer[["종목코드","종목명","시장","E_GRADE"]],S_test[["code","S_GRADE"]],right_on="code",left_on="종목코드",how="left").drop(["code"],axis=1)
final_answer=pd.merge(final_answer[["종목코드","종목명","시장","E_GRADE","S_GRADE"]],G_test[["code","G_GRADE"]],right_on="code",left_on="종목코드",how="left").drop(["code"],axis=1)
final_answer=pd.merge(final_answer[["종목코드","종목명","시장","E_GRADE","S_GRADE","G_GRADE"]],ESG_test[["code","ESG_GRADE"]],right_on="code",left_on="종목코드",how="left").drop(["code"],axis=1)

In [177]:
final_answer

,종목코드,종목명,시장,E_GRADE,S_GRADE,G_GRADE,ESG_GRADE
0,'005930',삼성전자,KOSPI,4,2,1,3
1,'207940',삼성바이오로직스,KOSPI,5,5,2,6
2,'051910',LG화학,KOSPI,4,2,1,2
3,'000270',기아,KOSPI,3,2,1,3
4,'035720',카카오,KOSPI,2,2,1,4
...,...,...,...,...,...,...,...
279,'090360',로보스타,KOSDAQ,9,9,5,9
280,'033170',시그네틱스,KOSDAQ,6,9,8,7
281,'060370',KT서브마린,KOSDAQ,8,8,5,9
282,'053350',이니텍,KOSDAQ,9,5,6,8


In [178]:
final_answer.to_csv("[ESG_러닝머신]ESG점수 예측 답안지.csv",encoding="utf-8",index=False)
final_answer.to_csv("[ESG_러닝머신]ESG점수 예측 답안지_ansi.csv",encoding="cp949",index=False)

## 6. 기타: 전체 기업 ESG_등급 얻기

In [179]:
environ_final=pd.read_csv("data/pipeline3/esg_environ_final_cat.csv")
govern_final=pd.read_csv("data/pipeline3/esg_govern_final_cat.csv")
social_final=pd.read_csv("data/pipeline3/esg_social_final_cat.csv")
esg_train_y=pd.read_csv("data/final_esg/esg_train_data.csv")
esg_test_y=pd.read_csv("data/final_esg/esg_test_data.csv")

In [180]:
environ_final=environ_final[['code','industry_med_code','raw', 'water', 'energy', 'air_pollu',
       'water_pollu', 'waste', 'chemical', 'water_reuse', 'waste_reuse',
       'exist', 'asset_sum_cat']]
social_final=social_final[['code', 'industry_med_code','reg_prob', 'female_prob',
       'reg_female_prob', 'female_money_prob', 'exist', 'disabled',
       'industry_accident', 'volunteer', 'asset_sum_cat']]
govern_final=govern_final[['code','industry_med_code','Outer_percentage',
       'female_percentage',  'vote_percentage',
       'attendance', 'op_uniformity', 'audit_num', 'exist', 
       'asset_sum_cat']]

In [181]:
ESG_final=pd.merge(environ_final,govern_final,on="code")
ESG_final=pd.merge(ESG_final,social_final,on="code")
ESG_final=ESG_final[['code','industry_med_code','asset_sum_cat','raw', 'water', 'energy', 'air_pollu',
       'water_pollu', 'waste', 'chemical', 'water_reuse', 'waste_reuse','Outer_percentage', 'female_percentage','vote_percentage', 'attendance', 'op_uniformity','audit_num','reg_female_prob', 'female_money_prob', 'exist', 'disabled',
       'industry_accident', 'volunteer','exist_x','exist_y']]
ESG_final["exist_total"]=ESG_final["exist"]+ESG_final["exist_x"]+ESG_final["exist_y"]
ESG_final=ESG_final.drop(["exist","exist_x","exist_y"],axis=1)

In [182]:
ESG_final['industry_med_code']=pd.to_numeric(ESG_final['industry_med_code'].str.replace("'",""))
ESG_final['industry_med_code']=ESG_final['industry_med_code'].fillna(0)
environ_final['industry_med_code']=pd.to_numeric(environ_final['industry_med_code'].str.replace("'",""))
environ_final['industry_med_code']=environ_final['industry_med_code'].fillna(0)
social_final['industry_med_code']=pd.to_numeric(social_final['industry_med_code'].str.replace("'",""))
social_final['industry_med_code']=social_final['industry_med_code'].fillna(0)
govern_final['industry_med_code']=pd.to_numeric(govern_final['industry_med_code'].str.replace("'",""))
govern_final['industry_med_code']=govern_final['industry_med_code'].fillna(0)

In [183]:
E_pred = E_RF_tune.predict(environ_final.drop(["code"],axis=1))
S_pred = S_RF_tune.predict(social_final.drop(["code"],axis=1))
G_pred = G_RF_tune.predict(govern_final.drop(["code"],axis=1))
ESG_pred = ESG_RF_tune.predict(ESG_final.drop(["code"],axis=1))

In [184]:
environ_final["E_GRADE"]=E_pred
social_final["S_GRADE"]=S_pred
govern_final["G_GRADE"]=G_pred
ESG_final["ESG_GRADE"]=ESG_pred

In [185]:
ESG_total=pd.concat([esg_train_y,esg_test_y],axis=0).reset_index(drop=True)

In [186]:
ESG_total=pd.merge(ESG_total[["종목코드","종목명","시장"]],environ_final[["code","E_GRADE"]],right_on="code",left_on="종목코드",how="left").drop(["code"],axis=1)
ESG_total=pd.merge(ESG_total[["종목코드","종목명","시장","E_GRADE"]],social_final[["code","S_GRADE"]],right_on="code",left_on="종목코드",how="left").drop(["code"],axis=1)
ESG_total=pd.merge(ESG_total[["종목코드","종목명","시장","E_GRADE","S_GRADE"]],govern_final[["code","G_GRADE"]],right_on="code",left_on="종목코드",how="left").drop(["code"],axis=1)
ESG_total=pd.merge(ESG_total[["종목코드","종목명","시장","E_GRADE","S_GRADE","G_GRADE"]],ESG_final[["code","ESG_GRADE"]],right_on="code",left_on="종목코드",how="left").drop(["code"],axis=1)

In [187]:
ESG_total

,종목코드,종목명,시장,E_GRADE,S_GRADE,G_GRADE,ESG_GRADE
0,'000660',SK하이닉스,KOSPI,4,1,2,3
1,'035420',NAVER,KOSPI,2,3,1,2
2,'005380',현대차,KOSPI,3,2,1,2
3,'006400',삼성SDI,KOSPI,5,3,4,4
4,'068270',셀트리온,KOSPI,8,7,3,7
...,...,...,...,...,...,...,...
942,'090360',로보스타,KOSDAQ,9,9,5,9
943,'033170',시그네틱스,KOSDAQ,6,9,8,7
944,'060370',KT서브마린,KOSDAQ,8,8,5,9
945,'053350',이니텍,KOSDAQ,9,5,6,8


In [188]:
ESG_total.to_csv("data/ESG_total_grade.csv",encoding="utf-8",index=False)

In [189]:
def get_esg_score():
    answer=pd.read_csv("data/ESG_total_grade.csv")
    return answer

In [190]:
get_esg_score()

,종목코드,종목명,시장,E_GRADE,S_GRADE,G_GRADE,ESG_GRADE
0,'000660',SK하이닉스,KOSPI,4,1,2,3
1,'035420',NAVER,KOSPI,2,3,1,2
2,'005380',현대차,KOSPI,3,2,1,2
3,'006400',삼성SDI,KOSPI,5,3,4,4
4,'068270',셀트리온,KOSPI,8,7,3,7
...,...,...,...,...,...,...,...
942,'090360',로보스타,KOSDAQ,9,9,5,9
943,'033170',시그네틱스,KOSDAQ,6,9,8,7
944,'060370',KT서브마린,KOSDAQ,8,8,5,9
945,'053350',이니텍,KOSDAQ,9,5,6,8
